In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.python.keras import utils
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, InputLayer
import keras
import imageio
from PIL import Image
import shutil

import sklearn as sk
import tensorflow as tf
from keras.applications.resnet50 import ResNet50

from sklearn.model_selection import train_test_split
import seaborn as sns
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

import torchvision.transforms as trasnforms
from torchvision import datasets
import torch
from matplotlib import pyplot as okt
import numpy as np


In [ ]:
train_file = "../input/plant-pathology-2020-fgvc7/train.csv"
folder =   "../input/plant-pathology-2020-fgvc7/images/"

path_img = "../input/plant-pathology-2020-fgvc7/images/"
sub_path = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"

path = "../input/plant-pathology-2020-fgvc7/"

test_file = "../input/plant-pathology-2020-fgvc7/test.csv"

In [ ]:
BATCH_SIZE = 8

In [ ]:
transform = trasnforms.Compose(
            [trasnforms.ToTensor(),
            trasnforms.Normalize(0.5, 0.5)])



In [ ]:

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sub = pd.read_csv(path + 'sample_submission.csv')


train_paths = train.image_id.apply(lambda x: path_img + x + '.jpg').values
test_paths = test.image_id.apply(lambda x: path_img + x + '.jpg').values
train_labels = train.loc[:, 'healthy':].values

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)

)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((test_paths))
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)

)

In [ ]:
print(train_dataset)


In [ ]:
# Look at one imnage 

dataiter = iter(train_dataset)
images, labels = dataiter.next()
images.shape, labels. shape

In [ ]:
plt.imshow(images[0])

**Create the model**

In [ ]:
def get_label(row):
    if row['healthy'] : 
        return 'healthy'
    elif row['multiple_diseases']:
        return 'multiple_diseases'
    elif row['rust']:
             return 'rust'
    elif row['scab']:
             return 'scab'

In [ ]:
df['label'] = df.apply(get_label, axis = 1)

In [ ]:
# df['img'] = train_file + 'images/' + df['image_id'].astype(str)+'.jpg'
df['file_name'] = df['image_id'].astype(str)+'.jpg'
# df_test['img'] = test_file + 'images/' + df_test['image_id'].astype(str)+'.jpg'
df_test['file_name'] = df_test['image_id'].astype(str)+'.jpg'

In [ ]:
df.head()

In [ ]:
df_train, df_validate = train_test_split(df, 
                                         test_size = .2, random_state = 42)

In [ ]:
print(f"Training Size : {len(df_train)}")
print(f"Validation Size : {len(df_validate)}")

In [ ]:
im = Image.open('../input/plant-pathology-2020-fgvc7/images/Train_1.jpg')
width, height = im.size
print(width, height)

In [ ]:
# Normalize the data

BATCH = 4
weidth = int(width / 1.5)
height = int(height/1.5)
train_datagen = ImageDataGenerator(
                rescale = 1.0 / 255,
                horizontal_flip = True,
                fill_mode = 'nearest' )

train_generator = train_datagen.flow_from_dataframe(
        dataframe = df_train,
        directory = path,
        x_col = 'file_name',
        y_col =  'label',    # [('healthy', 'multiple_diseases', 'rust', 'scab')],
        target_size = (height, width),
        batch_size = BATCH,
        class_mode = 'categorical',
        classes = ['healthy', 'multiple_diseases', 'rust', 'scab']
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe = df_validate,
        directory = path,
        x_col = 'file_name',
        y_col =  'label',    # [('healthy', 'multiple_diseases', 'rust', 'scab')],
        target_size = (height, width),
        batch_size = BATCH,
        class_mode = 'categorical',
        classes = ['healthy', 'multiple_diseases', 'rust', 'scab']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# import Resnet 50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(384,384,3)) 
len(base_model.layers)

In [ ]:
x= base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x) 
x=Dense(64,activation='relu')(x) 
preds=Dense(4,activation="softmax")(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = preds)

In [ ]:
# Inspect DATA
for x, y in train_generator:
    break
x.shape, y.shape

In [ ]:
plt.imshow(x[0])

In [ ]:
n_epochs = 20
valiation_steps = len(df_validate)
model.compile(loss = 'categorical_crossentropy', 
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [ ]:
history = model.fit(train_generator,
                    epochs=n_epochs,
                    # callbacks=[lr_schedule],
                    steps_per_epoch= len(val_generator), #20 ,
                    validation_data=val_generator,
                    validation_steps = len(val_generator))

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'Validation')
plt.legend()
plt.show()

In [ ]:
df_test.head()

In [ ]:
submit_datagen = ImageDataGenerator(rescale = 1. / 255)

submit_generator = submit_datagen.flow_from_dataframe(
            dataframe = df_test,
            directory = path,
            x_col = 'file_name',
            y_col =  None,    
            target_size = (height, width),
            batch_size = BATCH,
            class_mode = None,
            classes = ['healthy', 'multiple_diseases', 'rust', 'scab']
)            


y_pred = model.predict(submit_generator, steps = len(df_test))

In [ ]:
submit = pd.concat([df_test, pd.DataFrame(y_pred, columns=colnames) ],  axis = 1)

In [ ]:
submit = submit[['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab' ]]

In [ ]:
submit.head()

In [ ]:
submit.to_csv("/kaggle/working/submit.csv", index = False)